In [1]:
import pandas as pd 
import numpy as np 
import glob
from Utils_ETL import helper as hpe
from datetime import datetime

Para una muestra representativa, voy a tomar Texas ya que es donde están los locales más importantes de Texas Roadhouse, además de ser uno de los lugares dónde es más tradicional esta comida

In [2]:
df_reviews = hpe.merge_dataframes_ETL("datasets/review-Texas")

In [3]:
df_meta1 = hpe.merge_dataframes_ETL("datasets/meta1")

In [4]:
df_meta2 = hpe.merge_dataframes_ETL("datasets/meta2")

In [5]:
df_meta3 = hpe.merge_dataframes_ETL("datasets/meta3")

In [6]:
df_meta4 = hpe.merge_dataframes_ETL("datasets/meta4")

In [7]:
df_meta1 = df_meta1.explode("category")
df_meta2 = df_meta2.explode("category")
df_meta3 = df_meta3.explode("category")
df_meta4 = df_meta4.explode("category")

Vamos a ver que categorias tienen los restaurantes Texas Roadhouse y las anotamos

In [8]:
#guia para ver en que se basó la funcion
lista_categoria = []
mask = df_meta2["name"] == "Texas Roadhouse"
TexRod = df_meta2[mask]
categorias = TexRod.category.values
for categoria in categorias:
    if categoria in lista_categoria:
        pass
    else:
        lista_categoria.append(categoria)
lista_categoria

['Steak house']

In [9]:
valores1 = hpe.associated_values_other_columns(df_meta1,"name","Texas Roadhouse","category")
valores2 = hpe.associated_values_other_columns(df_meta2,"name","Texas Roadhouse","category")
valores3 = hpe.associated_values_other_columns(df_meta3,"name","Texas Roadhouse","category")
valores4 = hpe.associated_values_other_columns(df_meta4,"name","Texas Roadhouse","category")
valores = valores1 + valores2 + valores3 + valores4
valores = list(set(valores))
valores

['American restaurant',
 'New American restaurant',
 'Restaurant',
 'Bar',
 'Steak house']

Vamos a filtrar solamente a los locales que tengan esas categorias para, despues, quedarnos solamente con las reviews de lugares parecidos a Texas Roadhouse, por categoria

In [10]:
#guia para ver en que se basó la funcion
mask = df_meta4["category"].isin(valores)
meta_filtrado4 = df_meta4[mask]

In [11]:
meta_filtrado1 = hpe.filer_by_values_in_column(df_meta1,"category",valores)
meta_filtrado2 = hpe.filer_by_values_in_column(df_meta2,"category",valores)
meta_filtrado3 = hpe.filer_by_values_in_column(df_meta3,"category",valores)
meta_filtrado4 = hpe.filer_by_values_in_column(df_meta4,"category",valores)

In [12]:
meta_filtrado = pd.concat([meta_filtrado1, meta_filtrado2, meta_filtrado3,meta_filtrado4], ignore_index=True)

Estoy haciendo el ETL de reviews, no de los metadatos, por lo tanto me voy a quedar con las columnas que me sirver para ese proposito

In [13]:
metadata = meta_filtrado[["name","address","gmap_id","description","category","avg_rating","price"]]

Unimos los dataframes

In [14]:
df_reviews_meta = pd.merge(df_reviews, metadata, on='gmap_id', how='left')

In [15]:
df_reviews_meta

,user_id,name_x,time,rating,text,pics,resp,gmap_id,name_y,address,description,category,avg_rating,price
0,1.105453e+20,Kimberly Feger,1625431734826,5,"The pharmacist, Erin, is phenomenal. She was s...",None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
1,1.036198e+20,Briana Streit,1568336283913,2,I gave them 2 stars because they offer prescri...,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
2,1.019167e+20,Sylvia Caudillo,1519873586872,1,If I could put minus stars I would. This has t...,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
3,1.178709e+20,Ginger Kinyon,1571638384637,1,Please fix your restroom doors,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
4,1.103973e+20,Angeles Arellano,1546568877087,1,This pharmacy Walmart dose not work not come a...,None,None,0x864c3998b8d8dc83:0x57ffabe1e2322320,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430180,1.021681e+20,t williams,1562883151018,5,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN
2430181,1.054173e+20,Erika Lira,1532009672494,5,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN
2430182,1.107857e+20,Jon Brent,1567968578697,5,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN
2430183,1.091676e+20,lunna cabal,1506476840285,4,None,None,None,0x8640d215ff1e43e5:0xaeeb5211a25191f4,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#cambiamos los nombres confusos
df_reviews_meta = df_reviews_meta.rename(columns={'name_x': 'name',
                                                  "name_y": "local"})

Nos quedamos solo con los lugares con la misma categoria de nuestro cliente

In [17]:
df_reviews_meta_filtered =  hpe.null_filter_by_columns(df_reviews_meta,"category")

In [18]:
df_reviews_meta_filtered

,user_id,name,time,rating,text,pics,resp,gmap_id,local,address,description,category,avg_rating,price
57,1.089906e+20,Karina Asher,1630618611013,5,If you’ve seen Harold and Kumar Go To White Ca...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
58,1.089906e+20,Karina Asher,1630618611013,5,If you’ve seen Harold and Kumar Go To White Ca...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
59,1.166907e+20,Robert Stevens,1630859678572,5,Had the sliders and onion rings for the first ...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1630860556690, 'text': 'Thanks for yo...",0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
60,1.166907e+20,Robert Stevens,1630859678572,5,Had the sliders and onion rings for the first ...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1630860556690, 'text': 'Thanks for yo...",0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
61,1.113003e+20,Kyle Johnson,1630623576851,3,I appreciate Golden Castle trying something ne...,None,None,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",None,Restaurant,4.5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429635,1.156814e+20,Zoraida Vences,1611626536005,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
2429636,1.153003e+20,Gabriel Gonzales,1607396954553,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
2429637,1.009257e+20,Jacqueline Cano,1569606978949,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None
2429638,1.091067e+20,Manuel Martinez,1607657246694,5,None,None,None,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",None,Restaurant,4.8,None


In [19]:
df_reviews_meta_filtered = df_reviews_meta_filtered.drop(["pics","resp","price","description"], axis = 1)

In [20]:
df_reviews_meta_filtered.drop_duplicates(inplace=True)

In [21]:
def data_overview(df_Accesos_Cien):
    '''    
    Devuelve:
        un dataframe de pandas que tiene lo siguiente: 
        ** nombre_campo, tipo_datos,no_nulos_%,nulos_%,nulos(cantidad)
    '''

    result_dict = {"nombre_campo": [], "tipo_datos": [], "NO_nulos_%": [], "nulos_%": [], "nulos_cantidad": []}

    for column in df_Accesos_Cien.columns:
        porcentaje_no_nulos = (df_Accesos_Cien[column].count() / len(df_Accesos_Cien)) * 100
        result_dict["nombre_campo"].append(column)
        result_dict["tipo_datos"].append(df_Accesos_Cien[column].apply(type).unique())
        result_dict["NO_nulos_%"].append(round(porcentaje_no_nulos, 2))
        result_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        result_dict["nulos_cantidad"].append(df_Accesos_Cien[column].isnull().sum())

    df_Accesos_Cien_output = pd.DataFrame(result_dict)
        
    return df_Accesos_Cien_output

In [22]:
data_overview(df_reviews_meta_filtered)

,nombre_campo,tipo_datos,NO_nulos_%,nulos_%,nulos_cantidad
0,user_id,[<class 'float'>],100.00,0.00,0
1,name,[<class 'str'>],100.00,0.00,0
2,time,[<class 'int'>],100.00,0.00,0
3,rating,[<class 'int'>],100.00,0.00,0
4,text,"[<class 'str'>, <class 'NoneType'>]",56.72,43.28,203042
5,gmap_id,[<class 'str'>],100.00,0.00,0
6,local,[<class 'str'>],100.00,0.00,0
7,address,"[<class 'str'>, <class 'NoneType'>]",99.98,0.02,105
8,category,[<class 'str'>],100.00,0.00,0
9,avg_rating,[<class 'float'>],100.00,0.00,0


Vamos a llenar los vacios

In [23]:
df_reviews_meta_filtered.fillna("Sin Datos", inplace=True)

In [24]:
df_reviews_meta_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 469111 entries, 57 to 2429639
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     469111 non-null  float64
 1   name        469111 non-null  object 
 2   time        469111 non-null  int64  
 3   rating      469111 non-null  int64  
 4   text        469111 non-null  object 
 5   gmap_id     469111 non-null  object 
 6   local       469111 non-null  object 
 7   address     469111 non-null  object 
 8   category    469111 non-null  object 
 9   avg_rating  469111 non-null  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 39.4+ MB


Cambiamos la columna id por str y pasamos la columna time a un formato más claro

In [25]:
df_reviews_meta_filtered['user_id'] = df_reviews_meta_filtered['user_id'].astype(str)

In [26]:
df_reviews_meta_filtered['time'] = pd.to_datetime(df_reviews_meta_filtered['time'], unit='ms')

In [27]:
df_reviews_meta_filtered

,user_id,name,time,rating,text,gmap_id,local,address,category,avg_rating
57,1.0899060653234818e+20,Karina Asher,2021-09-02 21:36:51.013,5,If you’ve seen Harold and Kumar Go To White Ca...,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Restaurant,4.5
59,1.1669074312609345e+20,Robert Stevens,2021-09-05 16:34:38.572,5,Had the sliders and onion rings for the first ...,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Restaurant,4.5
61,1.1130031769106358e+20,Kyle Johnson,2021-09-02 22:59:36.851,3,I appreciate Golden Castle trying something ne...,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Restaurant,4.5
63,1.1445358016303402e+20,Toufic Chahin,2021-07-21 14:20:52.621,5,They've developed an idea and made it their ow...,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Restaurant,4.5
65,1.132307480700772e+20,Kyle Mitchell,2021-08-20 00:59:56.801,5,So yummy! Hot and fresh!,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Restaurant,4.5
...,...,...,...,...,...,...,...,...,...,...
2429635,1.1568135137372519e+20,Zoraida Vences,2021-01-26 02:02:16.005,5,Sin Datos,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",Restaurant,4.8
2429636,1.1530030066390175e+20,Gabriel Gonzales,2020-12-08 03:09:14.553,5,Sin Datos,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",Restaurant,4.8
2429637,1.0092570008285064e+20,Jacqueline Cano,2019-09-27 17:56:18.949,5,Sin Datos,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",Restaurant,4.8
2429638,1.0910670767081041e+20,Manuel Martinez,2020-12-11 03:27:26.694,5,Sin Datos,0x865ca94e2edeb237:0xdfc44b585273f4c5,Taquitos Mi Rancho,"Taquitos Mi Rancho, 651 E Hopkins St, San Marc...",Restaurant,4.8


In [28]:
df_reviews_meta_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 469111 entries, 57 to 2429639
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     469111 non-null  object        
 1   name        469111 non-null  object        
 2   time        469111 non-null  datetime64[ns]
 3   rating      469111 non-null  int64         
 4   text        469111 non-null  object        
 5   gmap_id     469111 non-null  object        
 6   local       469111 non-null  object        
 7   address     469111 non-null  object        
 8   category    469111 non-null  object        
 9   avg_rating  469111 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 39.4+ MB


In [30]:
df_reviews_meta_filtered.to_csv('datasets/df_reviews.csv', index=False)